----
# Cuaderno 5 - Regresión lineal y métricas de evaluación
## Ariel Palazzesi - 2026
----

En este cuaderno vamos a construir, entrenar y evaluar un modelo de **regresión lineal** utilizando un dataset real.

Vamos a trabajar paso a paso con el conjunto de datos del Titanic, aplicando lo aprendido en la clase teórica:

- ¿Qué es la regresión lineal y cuándo se utiliza?
- ¿Cómo se entrena un modelo utilizando Scikit-learn?
- ¿Qué métricas se utilizan para evaluar un modelo de regresión?
- ¿Cómo interpretar los resultados?

Este cuaderno está diseñado para ayudarte a unir estos conceptos con la práctica, y al mismo tiempo desarrollar criterio para analizar la calidad de un modelo predictivo.


## Carga del dataset

Vamos a seguir trabajando con el dataset del **Titanic**, que ya utilizamos en cuadernos anteriores. Para poder usarlo en este cuaderno, debés subir el archivo `Titanic-Dataset.csv` al entorno de ejecución de Colab.

1. Hacé clic en el ícono de carpeta (📁) en el panel izquierdo.
2. Subí el archivo `Titanic-Dataset.csv` desde tu computadora.
3. Verificá que aparezca en la ruta `/`.

Luego, ejecutá el siguiente bloque de código para importar las librerías necesarias y cargar el archivo.


In [1]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Estilo para gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# Cargamos el dataset
ruta = "Titanic-Dataset.csv"
df = pd.read_csv(ruta)

# Mostramos las primeras filas
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Selección de variables y preparación del dataset

Para aplicar un modelo de regresión lineal necesitamos una variable objetivo **numérica y continua**. En este caso, vamos a predecir el valor de la columna `Fare`, que representa la tarifa que pagó cada pasajero del Titanic.

Para eso, vamos a utilizar como variables predictoras las siguientes columnas:
- `Pclass` (clase del pasajero)
- `Sex` (género, codificado)
- `Age` (edad)
- `SibSp` (cantidad de hermanos o cónyuges a bordo)
- `Parch` (cantidad de padres o hijos a bordo)

Antes de entrenar el modelo, tenemos que:
- Codificar la variable `Sex`, que es categórica.
- Tratar los valores faltantes en `Age`.
- Verificar que no haya valores nulos en las columnas seleccionadas.


In [2]:
# Creamos una copia del dataset original
df_modelo = df.copy()

# Codificamos la variable 'Sex' (male = 0, female = 1)
df_modelo['Sex_encoded'] = df_modelo['Sex'].map({'male': 0, 'female': 1})

# Seleccionamos las columnas que vamos a usar
columnas = ['Fare', 'Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch']
df_modelo = df_modelo[columnas]

# Eliminamos filas con valores faltantes
df_modelo = df_modelo.dropna()

# Verificamos que no queden valores nulos
df_modelo.isna().sum()


Fare           0
Pclass         0
Sex_encoded    0
Age            0
SibSp          0
Parch          0
dtype: int64

## División del dataset y entrenamiento del modelo

Ahora que ya tenemos nuestros datos limpios y listos, vamos a dividir el conjunto en dos partes: una para **entrenar** el modelo (80%) y otra para **evaluarlo** (20%). Esta división nos permitirá comprobar qué tan bien funciona el modelo con datos que nunca vio durante el entrenamiento.

Luego, entrenaremos un modelo de **regresión lineal** utilizando `Scikit-learn`(*). Este modelo ajustará una recta multidimensional que intentará predecir el valor de la tarifa (`Fare`) a partir de las demás variables.

Finalmente, veremos qué coeficiente asignó el modelo a cada variable, lo cual nos permitirá interpretar la influencia que tiene cada una en la predicción.

---

*`¨Scikit-learn` es una biblioteca especializada en **aprendizaje automático clásico** que ofrece herramientas simples y consistentes para construir, entrenar y evaluar modelos de Machine Learning. Reúne en una misma librería algoritmos de clasificación, regresión, clustering y reducción de dimensionalidad, junto con utilidades para preparar datos, dividir conjuntos, aplicar validación cruzada y medir el rendimiento de los modelos. Todos los modelos siguen una interfaz común (métodos como `fit`, `predict` y `score`), lo que facilita aprender, comparar algoritmos y experimentar rápidamente. Es una de las bibliotecas más usadas tanto en el ámbito académico como profesional para prototipar soluciones de Machine Learning y establecer modelos base antes de pasar a enfoques más complejos.*


In [3]:
# Definimos las variables predictoras (X) y la variable objetivo (y)
X = df_modelo[['Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch']]
y = df_modelo['Fare']

# Dividimos el conjunto de datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos y entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Mostramos los coeficientes de cada variable
coeficientes = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': modelo.coef_
})
print("Intercepto:", modelo.intercept_)
coeficientes


Intercepto: 111.12250139059563


,Variable,Coeficiente
0,Pclass,-36.587785
1,Sex_encoded,4.315933
2,Age,-0.153418
3,SibSp,6.648888
4,Parch,11.593018


## MAE - Error Absoluto Medio

El **MAE** (*Mean Absolute Error* o Error Absoluto Medio) es una métrica utilizada en problemas de **regresión** para medir, en promedio, cuánto se equivoca un modelo en sus predicciones respecto de los valores reales. Su principal ventaja es que se expresa en las **mismas unidades que la variable objetivo** que estamos prediciendo (en este caso, `Fare`), lo que hace que su interpretación sea directa e intuitiva.

El MAE se calcula tomando la diferencia absoluta entre cada valor real y su predicción, y luego promediando todas esas diferencias. Al usar el valor absoluto, no importa si el error es positivo o negativo: todos los errores se consideran por igual. Esto significa que **cada error aporta el mismo peso al resultado final**, independientemente de su magnitud.

Esta característica hace que el MAE sea una métrica **estable y robusta**, útil cuando no queremos que unos pocos errores muy grandes influyan de manera desproporcionada en la evaluación del modelo. A diferencia de otras métricas, el MAE refleja de forma clara el error promedio esperado en una predicción individual.

En el código, la función `mean_absolute_error()` compara los valores reales del conjunto de prueba (`y_test`) con las predicciones generadas por el modelo mediante `modelo.predict(X_test)`. El resultado se almacena en la variable `mae` y luego se imprime en pantalla, mostrando el error promedio con dos decimales para facilitar su lectura.

El MAE nos permite así responder a una pregunta clave: **en promedio, ¿cuánto se equivoca el modelo cada vez que hace una predicción?**

In [4]:
mae = mean_absolute_error(y_test, modelo.predict(X_test))
print(f"MAE: {mae:.2f}")


MAE: 25.34


## Interpretación de resultados

El modelo entrenado muestra un **MAE de 25.34**, lo que significa que, en promedio, se equivoca por unos 25 unidades monetarias (en este caso, probablemente libras o dólares) al predecir el valor de la tarifa (`Fare`). El **MSE** es más alto, como era de esperarse, ya que penaliza los errores grandes.


## MSE - Error Cuadrático Medio

El **MSE** (*Mean Squared Error* o Error Cuadrático Medio) es una métrica utilizada en problemas de **regresión** que mide el promedio de los errores al cuadrado entre los valores reales y las predicciones del modelo. A diferencia del MAE, en este caso las diferencias no solo se consideran en valor absoluto, sino que además se **elevan al cuadrado** antes de promediarse.

Elevar los errores al cuadrado tiene un efecto importante: los **errores grandes pesan mucho más** que los errores pequeños. Esto hace que el MSE sea muy útil cuando queremos que el modelo sea muy sensible a predicciones con grandes desvíos, penalizando fuertemente esos casos.

Sin embargo, esta misma característica también representa una desventaja. Al elevar los errores al cuadrado, la métrica queda expresada en **unidades cuadráticas** (por ejemplo, pesos² o dólares²), lo que hace que su interpretación sea menos intuitiva que la del MAE. Por esta razón, el valor del MSE suele utilizarse más para **comparar modelos entre sí** que para interpretar el error en términos directos.

En el código, la función `mean_squared_error()` compara los valores reales del conjunto de prueba (`y_test`) con las predicciones generadas por el modelo mediante `modelo.predict(X_test)`. El resultado se guarda en la variable `mse` y luego se imprime en pantalla con dos decimales para facilitar su lectura.

El MSE nos permite evaluar no solo cuánto se equivoca el modelo en promedio, sino también **cuán graves son los errores grandes**.

In [5]:
mse = mean_squared_error(y_test, modelo.predict(X_test))
print(f"MSE: {mse:.2f}")


MSE: 3420.38


## ¿Qué significa un MSE de 3420,38?

El valor de **MSE = 3420,38** indica que el promedio de los **errores al cuadrado** entre las predicciones del modelo y los valores reales de la tarifa (`Fare`) es de 3.420,38.

Esto significa que el modelo está cometiendo errores de magnitud considerable, y que algunos de esos errores probablemente son bastante grandes. Como el MSE **penaliza más fuerte los errores grandes** (al elevarlos al cuadrado), este valor sugiere que hay casos en los que el modelo se aleja bastante de la realidad.

Si el rango de valores de `Fare` incluye montos bajos (como 7 u 8) y también montos muy altos (como 200 o más), este valor puede estar siendo afectado por **outliers** o casos extremos. Para interpretarlo mejor, conviene compararlo con el RMSE, que devuelve una medida más intuitiva en las mismas unidades que `Fare`.


## RMSE - Raíz del Error Cuadrático Medio

El **RMSE** (*Root Mean Squared Error* o Raíz del Error Cuadrático Medio) es una métrica utilizada en problemas de **regresión** que se obtiene tomando la raíz cuadrada del MSE. Al hacer esto, el error vuelve a expresarse en las **mismas unidades que la variable objetivo** que estamos prediciendo (en este caso, `Fare`), lo que facilita su interpretación.

Al igual que el MSE, el RMSE **penaliza con mayor fuerza los errores grandes**, ya que parte de los errores elevados al cuadrado antes de promediarlos. Sin embargo, al aplicar la raíz cuadrada, el valor final resulta más intuitivo y comparable con el MAE, ya que ambos se expresan en la misma escala que la variable original.

Por este motivo, el RMSE suele considerarse una **métrica intermedia** entre MAE y MSE: conserva la sensibilidad del MSE frente a grandes desvíos, pero evita el problema de las unidades cuadráticas. En muchos contextos prácticos, el RMSE se utiliza como una métrica principal para evaluar el rendimiento de modelos de regresión.

En el código, primero se calcula el MSE utilizando la función `mean_squared_error()`, comparando los valores reales del conjunto de prueba (`y_test`) con las predicciones del modelo (`modelo.predict(X_test)`). Luego, se aplica `np.sqrt()` para obtener la raíz cuadrada de ese valor. El resultado se almacena en la variable `rmse` y se muestra en pantalla con dos decimales.

El RMSE permite responder a una pregunta importante: **¿cuál es el error típico del modelo, penalizando especialmente los errores grandes, pero expresado en unidades comprensibles?**

In [6]:
# Calculamos RMSE tomando la raíz cuadrada del MSE
rmse = np.sqrt(mean_squared_error(y_test, modelo.predict(X_test)))
print(f"RMSE: {rmse:.2f}")


RMSE: 58.48


## ¿Qué significa un RMSE de 58,48?

El valor de **RMSE = 58,48** nos indica que, en promedio, el modelo se desvía unos **58 unidades monetarias** al predecir el valor de la tarifa (`Fare`) de un pasajero.

Como esta métrica se expresa en las mismas unidades que la variable objetivo, es fácil de interpretar: si muchas tarifas en el Titanic oscilaban entre 7 y 80, este error puede considerarse **moderado**, pero sugiere que aún hay margen para mejorar la precisión del modelo.

El RMSE también nos permite comparar distintos modelos de forma directa: cuanto **más bajo sea este valor**, mejor será la capacidad del modelo para hacer predicciones cercanas a los valores reales, especialmente cuando se combinan varias variables predictoras.


## R² - Coeficiente de Determinación

El **R²** (*coeficiente de determinación*) es una métrica utilizada en problemas de **regresión** que indica qué proporción de la variabilidad de la variable objetivo puede ser explicada por el modelo. En otras palabras, mide **qué tan bien el modelo logra capturar la relación entre las variables de entrada y el valor que se desea predecir**.

El valor de R² suele tomar valores entre 0 y 1. Un valor cercano a 1 indica que el modelo explica gran parte de la variabilidad de los datos, mientras que un valor cercano a 0 indica que el modelo explica muy poco, funcionando apenas mejor que una predicción basada en el promedio. En algunos casos, el R² puede ser negativo, lo que significa que el modelo se desempeña peor que una predicción constante basada en la media de la variable objetivo.

A diferencia de métricas como MAE, MSE o RMSE, el R² **no mide el tamaño del error**, sino la capacidad explicativa del modelo. Por este motivo, se utiliza principalmente para tener una **visión global del ajuste del modelo**, más que para interpretar errores individuales.

En el código, la función `r2_score()` compara los valores reales del conjunto de prueba (`y_test`) con las predicciones generadas por el modelo (`modelo.predict(X_test)`). El resultado se almacena en la variable `r2` y se imprime con cuatro decimales para facilitar la comparación entre modelos.

El R² permite responder a la pregunta: **¿qué porcentaje de la variación observada en los datos logra explicar el modelo?**
Por esta razón, suele utilizarse junto con otras métricas de error, ya que por sí sola no brinda una evaluación completa del rendimiento del modelo.

In [7]:
r2 = r2_score(y_test, modelo.predict(X_test))
print(f"R²: {r2:.4f}")


R²: 0.1784


## ¿Qué significa un R² de 0.1784?

El valor de **R² = 0.1784** indica que el modelo de regresión lineal logra explicar aproximadamente el **17,84% de la variabilidad** en las tarifas (`Fare`) pagadas por los pasajeros del Titanic.

Esto significa que el modelo **no está captando con demasiada eficacia** las relaciones entre las variables predictoras y el valor objetivo. Si bien no es un valor nulo, sigue siendo bajo: hay una gran parte del comportamiento de la variable `Fare` que no está siendo explicada por el modelo.

Esto puede deberse a varias razones: tal vez las variables utilizadas no son las más representativas para predecir la tarifa, o quizás la relación entre las variables y el precio no es lineal. Este valor nos invita a revisar el conjunto de variables elegidas, incorporar nuevas fuentes de información o considerar modelos más complejos.

El R² no mide el error en sí mismo, sino **cuánto del fenómeno estamos siendo capaces de explicar**. En este caso, el modelo capta una parte, pero deja bastante afuera.



## Comparación de métricas: MAE, MSE, RMSE y R²

Al evaluar modelos de regresión, no existe una única métrica “correcta”. Cada métrica aporta una mirada distinta sobre el rendimiento del modelo, y su utilidad depende del tipo de problema que se esté resolviendo y de qué aspectos del error sean más relevantes.

El **MAE** (Error Absoluto Medio) mide el error promedio en las mismas unidades que la variable objetivo. Es fácil de interpretar y trata a todos los errores por igual, por lo que resulta especialmente útil cuando se busca una medida clara del error típico del modelo y no se desea penalizar en exceso los errores grandes.

El **MSE** (Error Cuadrático Medio) eleva los errores al cuadrado antes de promediarlos, lo que hace que los errores grandes tengan un peso mucho mayor. Esta métrica es útil cuando los desvíos importantes son especialmente problemáticos y se quiere que el modelo sea muy sensible a ellos, aunque su interpretación es menos intuitiva debido a que queda expresada en unidades cuadráticas.

El **RMSE** (Raíz del Error Cuadrático Medio) combina características del MAE y del MSE. Al aplicar la raíz cuadrada al MSE, recupera las unidades originales de la variable objetivo, lo que facilita su interpretación, pero conserva la penalización fuerte de los errores grandes. Por este motivo, es una de las métricas más utilizadas en la práctica para evaluar modelos de regresión.

El **R²** (Coeficiente de Determinación) no mide el error en sí, sino la proporción de la variabilidad de la variable objetivo que el modelo logra explicar. Es útil para tener una visión global del ajuste del modelo, pero no indica cuánto se equivoca en términos absolutos. Por esta razón, suele usarse como complemento de otras métricas y no como única medida de desempeño.

### ¿Qué métrica elegir?

* Si buscás **una interpretación simple y directa del error**, el MAE es una buena opción.

* Si querés **penalizar fuertemente los errores grandes**, el MSE o el RMSE son más adecuados.

* Si necesitás una métrica **interpretables en las mismas unidades que la variable**, el RMSE suele ser preferible al MSE.

* Si querés evaluar **qué tan bien el modelo explica los datos en general**, el R² aporta una visión complementaria.

En la práctica, lo más recomendable es **utilizar varias métricas en conjunto**, ya que cada una revela un aspecto distinto del comportamiento del modelo.

## Conclusión y cierre

Construimos nuestro primer modelo de regresión lineal completo: desde la selección de variables hasta la evaluación con distintas métricas. Utilizamos el conjunto de datos del Titanic para predecir la tarifa (`Fare`) que pagó cada pasajero en función de sus características.

Los resultados obtenidos nos dejan un aprendizaje valioso. Con un **MAE de 25,34**, podemos decir que el modelo se equivoca en promedio unos 25 unidades monetarias, lo cual es razonable en algunos contextos. Sin embargo, un **MSE de 3420,38** y un **RMSE de 58,48** nos advierten que existen errores considerables en ciertas predicciones, lo que podría estar influenciado por valores atípicos o relaciones no capturadas por un modelo lineal.

Finalmente, el valor de **R² = 0,1784** nos confirma que el modelo solo logra explicar una pequeña parte de la variabilidad de la tarifa: el 17,84%.

Esto no significa que el modelo sea inútil, sino que **hay margen de mejora**, ya sea mediante la selección de otras variables, la incorporación de nuevas fuentes de datos o el uso de algoritmos más complejos. La regresión lineal es un excelente punto de partida para comprender cómo se entrena, interpreta y evalúa un modelo de predicción numérica.